In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import SGD
# tf.Keras utilis function
from tensorflow.keras.utils import to_categorical

(train_data, train_label), (test_data,test_label) = mnist.load_data()
print("共有 %d 訓練資料，每張圖的大小為 %d x %d" %train_data.shape)
print("共有 %d 測試資料，每張圖的大小為 %d x %d" %test_data.shape)

共有 60000 訓練資料，每張圖的大小為 28 x 28
共有 10000 測試資料，每張圖的大小為 28 x 28


In [2]:
train_data_first5 = train_data[train_label<5]
train_label_first5 = train_label[train_label<5]
test_data_first5 = test_data[test_label<5]
test_label_first5 = test_label[test_label<5]
print(train_data_first5.shape)
print(train_label_first5.shape)
train_data_last5 = train_data[train_label>=5]
# 將標籤 5-9 改為 0-4
train_label_last5 = train_label[train_label>=5]-5
test_data_last5 = test_data[test_label>=5]
test_label_last5 = test_label[test_label>=5]-5

(30596, 28, 28)
(30596,)


In [3]:
# 資料前處理與維度轉換
def preprocess(x):
    x = tf.cast(x, dtype = tf.float32)/255
    x = tf.reshape(x,[x.shape[0],28,28,1])
    return x

train_data_first5 = preprocess(train_data_first5)
test_data_first5 = preprocess(test_data_first5)
train_data_last5 = preprocess(train_data_last5)
test_data_last5 = preprocess(test_data_last5)

In [4]:
train_label_first5 = to_categorical(train_label_first5,5)
test_label_first5 = to_categorical(test_label_first5,5)
train_label_last5 = to_categorical(train_label_last5,5)
test_label_last5 = to_categorical(test_label_last5,5)
print(train_label_first5.shape)

(30596, 5)


In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3),
                 input_shape=(28,28,1),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, kernel_size = (3,3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dense(5, activation = "softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                25664     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 3

In [6]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam",
             metrics=["accuracy"])
history = model.fit(train_data_first5,train_label_first5,
                    validation_split = 0.2,
                    epochs=5, batch_size = 64,verbose=2)

Epoch 1/5
383/383 - 21s - loss: 0.1595 - accuracy: 0.9574 - val_loss: 0.0396 - val_accuracy: 0.9889
Epoch 2/5
383/383 - 1s - loss: 0.0346 - accuracy: 0.9899 - val_loss: 0.0242 - val_accuracy: 0.9923
Epoch 3/5
383/383 - 1s - loss: 0.0238 - accuracy: 0.9927 - val_loss: 0.0185 - val_accuracy: 0.9948
Epoch 4/5
383/383 - 1s - loss: 0.0170 - accuracy: 0.9948 - val_loss: 0.0222 - val_accuracy: 0.9933
Epoch 5/5
383/383 - 1s - loss: 0.0142 - accuracy: 0.9954 - val_loss: 0.0135 - val_accuracy: 0.9954


In [7]:
print(len(model.layers))
for i in range(len(model.layers)):
    print(i, model.layers[i])

7
0 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001982DBF3F10>
1 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000198310AF970>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000198310C49A0>
3 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000019831191760>
4 <tensorflow.python.keras.layers.core.Flatten object at 0x000001983119ED30>
5 <tensorflow.python.keras.layers.core.Dense object at 0x000001983117B2B0>
6 <tensorflow.python.keras.layers.core.Dense object at 0x000001983119EF10>


In [8]:
for i in range(4):
    model.layers[i].trainable = False

In [9]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam",
             metrics=["accuracy"])
history = model.fit(train_data_last5,train_label_last5,
                    validation_split = 0.2,
                    epochs=5, batch_size = 64,verbose=2)

Epoch 1/5
368/368 - 6s - loss: 0.2973 - accuracy: 0.9182 - val_loss: 0.0812 - val_accuracy: 0.9750
Epoch 2/5
368/368 - 1s - loss: 0.0591 - accuracy: 0.9813 - val_loss: 0.0549 - val_accuracy: 0.9830
Epoch 3/5
368/368 - 1s - loss: 0.0425 - accuracy: 0.9873 - val_loss: 0.0457 - val_accuracy: 0.9854
Epoch 4/5
368/368 - 1s - loss: 0.0344 - accuracy: 0.9893 - val_loss: 0.0415 - val_accuracy: 0.9876
Epoch 5/5
368/368 - 1s - loss: 0.0276 - accuracy: 0.9915 - val_loss: 0.0362 - val_accuracy: 0.9889


In [10]:
loss, accuracy = model.evaluate(test_data_last5,test_label_last5)

152/152 [==============================] - 5s 17ms/step - loss: 0.0246 - accuracy: 0.9926
